In [6]:
# @name graph_v3.2_v20190616
# @description notebook to build the NGLY1 Deficiency review knowledge graph v3.2
# @author Núria Queralt Rosinach
# @date 16 June 2019

# Description

This is the notebook for the creation of the first review network and derived hypotheses. 

* Using intermediary variables from workflow objects. In this workflow variables are directly used for the next step. 


* Review network: From Monarch knowledge graph, we built a network seeded by 8 nodes, retrieving their explicit relationships and all the relationships among all these nodes. Seed nodes:

    - 'MONDO:0007739' HD
    - 'HGNC:4851' Htt
    - 'CHEBI:18248' Iron (not working)
    - 'HGNC:18229' Rhes (RASD2)
    
Possible seed nodes:
https://monarchinitiative.org/search/Iron
* Connecting paths: query templates.

In [1]:
import transcriptomics, regulation, curation, monarch, graph, neo4jlib, hypothesis, summary, utils
import pandas as pd

In [ ]:
monarch.get_connections()

In [9]:
nodes, edges = monarch.get_neighbours(["HGNC:18229"])

100%|██████████| 1/1 [00:03<00:00,  3.09s/it]


In [10]:
len(nodes)

66

In [6]:
nodes

{'ENSEMBL:ENSP00000220592',
 'ENSEMBL:ENSP00000236995',
 'ENSEMBL:ENSP00000246314',
 'ENSEMBL:ENSP00000258682',
 'ENSEMBL:ENSP00000261793',
 'ENSEMBL:ENSP00000268712',
 'ENSEMBL:ENSP00000272163',
 'ENSEMBL:ENSP00000277853',
 'ENSEMBL:ENSP00000282356',
 'ENSEMBL:ENSP00000296402',
 'ENSEMBL:ENSP00000301624',
 'ENSEMBL:ENSP00000301948',
 'ENSEMBL:ENSP00000302967',
 'ENSEMBL:ENSP00000306759',
 'ENSEMBL:ENSP00000307082',
 'ENSEMBL:ENSP00000309591',
 'ENSEMBL:ENSP00000313950',
 'ENSEMBL:ENSP00000315599',
 'ENSEMBL:ENSP00000326375',
 'ENSEMBL:ENSP00000326427',
 'ENSEMBL:ENSP00000326518',
 'ENSEMBL:ENSP00000326544',
 'ENSEMBL:ENSP00000326900',
 'ENSEMBL:ENSP00000336783',
 'ENSEMBL:ENSP00000338371',
 'ENSEMBL:ENSP00000339004',
 'ENSEMBL:ENSP00000339740',
 'ENSEMBL:ENSP00000339883',
 'ENSEMBL:ENSP00000347184',
 'ENSEMBL:ENSP00000349467',
 'ENSEMBL:ENSP00000350028',
 'ENSEMBL:ENSP00000351542',
 'ENSEMBL:ENSP00000353622',
 'ENSEMBL:ENSP00000357621',
 'ENSEMBL:ENSP00000358659',
 'ENSEMBL:ENSP000003

In [5]:
nodes2, edges2 = monarch.get_neighbours(nodes)

100%|██████████| 152/152 [10:32<00:00,  2.37s/it]


## Edges library
### Review edges to integrate into the knowledge graph and prepare them as individual networks

#### TRANSCRIPTOMICS NETWORK
#### import transcriptomics
We retrieved edges from RNA-seq transcriptomics profiles using the `transcriptomics` module:

    - Experimental data sets: from Chow et al. paper [pmid:29346549] (NGLY1 deficiency model on fruit fly)

In [13]:
%%time
# prepare data to graph schema
csv_path = './transcriptomics/GSE64810_mlhd_DESeq2_diffexp_DESeq2_outlier_trimmed_adjust.txt'
data = transcriptomics.read_data(csv_path)


The function "read_data()" is running...

* This is the size of the raw expression data structure: (28087, 10)
* These are the expression attributes: Index(['Unnamed: 0', 'symbol', 'baseMean', 'HD.mean', 'Control.mean',
       'log2FoldChange', 'lfcSE', 'stat', 'pvalue', 'padj'],
      dtype='object')
* This is the first record:
           Unnamed: 0 symbol  baseMean    HD.mean  Control.mean  \
0  ENSG00000069011.10  PITX1  5.645675  18.684286      0.323793   

   log2FoldChange     lfcSE      stat        pvalue          padj  
0        4.769658  0.366367  13.01879  9.567529e-39  2.687232e-34  

The raw data is saved at: /home/karolis/Structured review/bioknowledge-reviewer/bioknowledge_reviewer/transcriptomics/HD/data/GSE64810_mlhd_DESeq2_diffexp_DESeq2_outlier_trimmed_adjust.csv


Finished read_data().

CPU times: user 155 ms, sys: 16.3 ms, total: 171 ms
Wall time: 166 ms


In [10]:
%%time
# prepare data to graph schema
csv_path = './transcriptomics/GSE64810_mlhd_DESeq2_diffexp_DESeq2_outlier_trimmed_adjust.txt'
data = transcriptomics.read_data(csv_path)
clean_data = transcriptomics.clean_data(data)
data_edges = transcriptomics.prepare_data_edges(clean_data)
rna_network = transcriptomics.prepare_rna_edges(data_edges)

# build network with graph schema
rna_edges = transcriptomics.build_edges(rna_network)
rna_nodes = transcriptomics.build_nodes(rna_network)


The function "read_data()" is running...

* This is the size of the raw expression data structure: (28087, 10)
* These are the expression attributes: Index(['Unnamed: 0', 'symbol', 'baseMean', 'HD.mean', 'Control.mean',
       'log2FoldChange', 'lfcSE', 'stat', 'pvalue', 'padj'],
      dtype='object')
* This is the first record:
           Unnamed: 0 symbol  baseMean    HD.mean  Control.mean  \
0  ENSG00000069011.10  PITX1  5.645675  18.684286      0.323793   

   log2FoldChange     lfcSE      stat        pvalue          padj  
0        4.769658  0.366367  13.01879  9.567529e-39  2.687232e-34  

The raw data is saved at: /home/karolis/Structured review/bioknowledge-reviewer/bioknowledge_reviewer/transcriptomics/HD/data/GSE64810_mlhd_DESeq2_diffexp_DESeq2_outlier_trimmed_adjust.csv


Finished read_data().


The function "clean_data()" is running. Keeping only data with FC > 1.5 and FDR < 5% ...

* This is the size of the clean expression data structure: (3209, 6)
* These are the clean 

- Transcriptomics network is returned as both digital object (`rna_edges`, `rna_nodes`) and CSV files at _**graph/**_ (`rna_edges_version.csv`, `rna_nodes_version.csv`)

In [3]:
# print type of objects
print('type edges:', type(rna_edges))
print('type nodes:', type(rna_nodes))
print()

# print objects sizes
print('len edges:', len(rna_edges))
print('len nodes:', len(rna_nodes))
print()

# print object attribute
print('attribute edges:', rna_edges[0].keys())
print('attribute nodes:', rna_nodes[0].keys())

type edges: <class 'list'>
type nodes: <class 'list'>

len edges: 386
len nodes: 386

attribute edges: dict_keys(['subject_id', 'object_id', 'property_id', 'property_label', 'property_description', 'property_uri', 'reference_uri', 'reference_supporting_text', 'reference_date'])
attribute nodes: dict_keys(['id', 'semantic_groups', 'preflabel', 'name', 'synonyms', 'description'])


#### REGULATION NETWORK
#### import regulation

We retrieved human TF gene expression regulation edges from several sources using the `regulation` module.

In [11]:
%%time
# prepare msigdb data
gmt_path = './regulation/msigdb/data/c3.tft.v6.1.entrez.gmt'
regulation.prepare_msigdb_data(gmt_path)

# prepare individual networks
data = regulation.load_tf_gene_edges()
dicts = regulation.get_gene_id_normalization_dictionaries(data)
data_edges = regulation.prepare_data_edges(data, dicts)

# prepare regulation network
reg_network = regulation.prepare_regulation_edges(data_edges)

# build network with graph schema
reg_edges = regulation.build_edges(reg_network)
reg_nodes = regulation.build_nodes(reg_network)


The function "prepare_msigdb_data()" is running...

* Number of Transcription Factor Targets (TFT) gene sets: 615

The MSigDB raw network is saved at: /home/karolis/Structured review/bioknowledge-reviewer/bioknowledge_reviewer/regulation/msigdb/out/tf_genelist_entrez_msigdb.json. Other reporting files are also saved at the same directory.


Finished prepare_msigdb_data().


The function "load_tf_gene_edges()" is running...

Finished load_tf_gene_edges().


The function "get_gene_id_normalization_dictionaries()" is running...

* Querying BioThings to map gene symbols to HGNC and Entrez IDs...
querying 1-1000...done.
querying 1001-2000...done.
querying 2001-3000...done.
querying 3001-3071...done.
Finished.
48 input query terms found no hit:
	['MYCMAX', 'AP1FJ', 'TAL1BETAITF2', 'HAND1E47', 'NKX62', 'GNCF', 'COMP1', 'AHRARNT', 'E2F1DP1RB', 'T
Pass "returnall=True" to return complete lists of duplicate or missing query terms.

Saving not found gene symbols at: /home/karolis/Structured revi

- Regulation network is returned as both digital object (`reg_edges`, `reg_nodes`) and CSV files at _**graph/**_ (`regulation_edges_version.csv`, `regulation_nodes_version.csv`)

In [5]:
# print type of objects
print('type edges:', type(reg_edges))
print('type nodes:', type(reg_nodes))
print()

# print objects sizes
print('len edges:', len(reg_edges))
print('len nodes:', len(reg_nodes))
print()

# print object attribute
print('attribute edges:', reg_edges[0].keys())
print('attribute nodes:', reg_nodes[0].keys())

type edges: <class 'list'>
type nodes: <class 'list'>

len edges: 197267
len nodes: 16966

attribute edges: dict_keys(['subject_id', 'object_id', 'property_id', 'property_label', 'property_description', 'property_uri', 'reference_uri', 'reference_supporting_text', 'reference_date'])
attribute nodes: dict_keys(['id', 'semantic_groups', 'preflabel', 'name', 'synonyms', 'description'])


#### CURATED NETWORK
#### import curation

We retrieved and prepared curated edges using the `curation` module. 

In [19]:
curation_edges = pd.read_csv("curation/data/HD/Empty_edges.csv")
curation_nodes = pd.read_csv("curation/data/HD/Empty_nodes.csv")

In [2]:
%%time
# graph v3.2
# read network from drive and concat all curated statements
curation_edges, curation_nodes = curation.read_network(version='v20180118')

# prepare data edges and nodes
data_edges = curation.prepare_data_edges(curation_edges)
data_nodes = curation.prepare_data_nodes(curation_nodes)

# prepare curated edges and nodes
curated_network = curation.prepare_curated_edges(data_edges)
curated_concepts = curation.prepare_curated_nodes(data_nodes)


# build edges and nodes files
curation_edges = curation.build_edges(curated_network)
curation_nodes = curation.build_nodes(curated_concepts)


The function "read_network()" is running...

Reading and concatenating all curated statements in the network...

* Curation edge files concatenated shape: (322, 22)

Reading and concatenating all curated nodes in the network...

* Curation node files concatenated shape: (318, 9)

Finished read_network().


The function "prepare_data_edges()" is running...

Preparing curated network...

Saving curated network at curation/...

*Curated edges data structure shape: (321, 9)
*Curated edges data structure fields: Index(['subject_id', 'property_id', 'object_id', 'reference_uri',
       'reference_supporting_text', 'reference_date', 'property_label',
       'property_description', 'property_uri'],
      dtype='object')

The curated edges are saved at: /home/karolis/Structured review/bioknowledge-reviewer/bioknowledge_reviewer/curation/curated_edges_v2020-11-16.csv


Finished prepare_data_edges().


The function "prepare_data_nodes()" is running...

Preparing curated nodes...

Saving curated n

KeyError: 'property_label'

- Curated network is returned as both digital object (`curation_edges`, `curation_nodes`) and CSV files at _**graph/**_ (`curated_graph_edges_version.csv`, `curated_graph_nodes_version.csv`)
- The original curated network, i.e. without graph data model normalization, is saved as CSV files at _**curation/**_ (`curated_edges_version.csv`, `curated_nodes_version.csv`)

In [7]:
# print type of objects
print('type edges:', type(curation_edges))
print('type nodes:', type(curation_nodes))
print()

# print objects sizes
print('len edges:', len(curation_edges))
print('len nodes:', len(curation_nodes))
print()

# print object attribute
print('attribute edges:', curation_edges[0].keys())
print('attribute nodes:', curation_nodes[0].keys())

type edges: <class 'list'>
type nodes: <class 'list'>

len edges: 362
len nodes: 302

attribute edges: dict_keys(['subject_id', 'object_id', 'property_id', 'property_label', 'property_description', 'property_uri', 'reference_uri', 'reference_supporting_text', 'reference_date', 'g2p_mark'])
attribute nodes: dict_keys(['id', 'semantic_groups', 'preflabel', 'name', 'synonyms', 'description'])


#### MONARCH NETWORK
#### import monarch
We retrieved edges from Monarch using the `monarch` module.

Tasks:

- From 8 seed nodes we retrieved 1st shell nodes
- From all seed and 1st shell nodes we retrieved ortho-phenotypes
- We retrieved extra edges among all of them, i.e. extra connectivity between: seed, 1st shell, ortholog-phenotype nodes

In [2]:
%%time
# prepare data to graph schema
# seed nodes
seedList = [ 
    'MONDO:0007739', # HD
    'HGNC:4851', # Htt
    'HGNC:182293', # Rhes
    'REACT:R-HSA-917937' #Iron uptake pathway
] 

# get first shell of neighbours
neighboursList = monarch.get_neighbours_list(seedList)
print(len(neighboursList))
# get second degree of neighbours
monarch_network = monarch.get_connections(neighboursList)
print(len(monarch_network))


The function "get_neighbours_list()" is running. Its runtime may take some minutes. If you interrupt the process, you will lose all the nodes retrieved and you should start over the execution of this function.


100%|██████████| 4/4 [00:29<00:00,  6.43s/it]



Finished get_neighbours_list().

1022


100%|██████████| 1022/1022 [1:10:20<00:00,  7.38s/it]


12221
CPU times: user 2min 4s, sys: 7.55 s, total: 2min 11s
Wall time: 1h 10min 49s


In [3]:
monarch.print_network(monarch_network, 'monarch_connections')

# build network with graph schema 
monarch_edges = monarch.build_edges(monarch_network)
monarch_nodes = monarch.build_nodes(monarch_network)


Saving Monarch edges at: '/home/karolis/Structured review/bioknowledge-reviewer/bioknowledge_reviewer/monarch/monarch_connections_v2020-11-20.csv'...


The function "build_edges()" is running...
df (12221, 9)

* This is the size of the edges file data structure: (12221, 9)
* These are the edges attributes: Index(['object_id', 'property_description', 'property_id', 'property_label',
       'property_uri', 'reference_date', 'reference_supporting_text',
       'reference_uri', 'subject_id'],
      dtype='object')
* This is the first record:
    object_id property_description property_id property_label  \
0  HP:0000739                   NA  RO:0002200  has phenotype   

                                property_uri reference_date  \
0  http://purl.obolibrary.org/obo/RO_0002200             NA   

                           reference_supporting_text reference_uri  subject_id  
0  This edge comes from the Monarch Knowledge Gra...            NA  HGNC:11007  

The Monarch network edges are buil

In [4]:
# print type of objects
print('type edges:', type(monarch_edges))
print('type nodes:', type(monarch_nodes))
print()

# print objects sizes
print('len edges:', len(monarch_edges))
print('len nodes:', len(monarch_nodes))
print()

# print object attribute
print('attribute edges:', monarch_edges[0].keys())
print('attribute nodes:', monarch_nodes[0].keys())

type edges: <class 'list'>
type nodes: <class 'list'>

len edges: 12221
len nodes: 734

attribute edges: dict_keys(['subject_id', 'object_id', 'property_id', 'property_label', 'property_description', 'property_uri', 'reference_uri', 'reference_supporting_text', 'reference_date'])
attribute nodes: dict_keys(['id', 'semantic_groups', 'preflabel', 'name', 'synonyms', 'description'])


In [17]:
%%time
# prepare data to graph schema
# seed nodes
seedList = [ 
    'MONDO:0007739', # HD
    'HGNC:4851', # Htt
    'HGNC:182293', # Rhes
] 

# get first shell of neighbours
neighboursList = monarch.get_neighbours_list(seedList)
print(len(neighboursList))

# introduce animal model ortho-phenotypes for seed and 1st shell neighbors
## For seed nodes:
seed_orthophenoList = monarch.get_orthopheno_list(seedList)
print(len(seed_orthophenoList))
## For 1st shell nodes:
neighbours_orthophenoList = monarch.get_orthopheno_list(neighboursList)
print(len(neighbours_orthophenoList))

# network nodes: seed + 1shell + ortholog-phentoype
geneList = sum([seedList,
                neighboursList,
                #seed_orthophenoList,
                #neighbours_orthophenoList
               ], 
               [])
print('genelist: ',len(geneList))

# get Monarch network
monarch_network = monarch.extract_edges(geneList)
print('network: ',len(monarch_network))

# save edges
monarch.print_network(monarch_network, 'monarch_connections')

# build network with graph schema 
monarch_edges = monarch.build_edges(monarch_network)
monarch_nodes = monarch.build_nodes(monarch_network)


The function "get_neighbours_list()" is running. Its runtime may take some minutes. If you interrupt the process, you will lose all the nodes retrieved and you should start over the execution of this function.


100%|██████████| 3/3 [00:18<00:00,  5.14s/it]



Finished get_neighbours_list().

648

The function "get_orthopheno_list()" is running. Its runtime may take some hours. If you interrupt the process, you will lose all the nodes retrieved and you should start over the execution of this function.


100%|██████████| 22/22 [01:08<00:00,  3.05s/it]



Finished get_orthopheno_list().

73

The function "get_orthopheno_list()" is running. Its runtime may take some hours. If you interrupt the process, you will lose all the nodes retrieved and you should start over the execution of this function.


100%|██████████| 2679/2679 [2:15:54<00:00,  2.81s/it]  



Finished get_orthopheno_list().

5128
genelist:  651

The function "extract_edges()" is running. Its runtime may take some hours. If you interrupt the process, you will lose all the edges retrieved and you should start over the execution of this function.


100%|██████████| 651/651 [36:19<00:00,  3.48s/it]  



Finished extract_edges(). To save the retrieved Monarch edges use the function "print_network()".

network:  3095

Saving Monarch edges at: '/home/karolis/Structured review/bioknowledge-reviewer/bioknowledge_reviewer/monarch/monarch_connections_v2020-11-15.csv'...


The function "build_edges()" is running...
df (3095, 9)

* This is the size of the edges file data structure: (3095, 9)
* These are the edges attributes: Index(['object_id', 'property_description', 'property_id', 'property_label',
       'property_uri', 'reference_date', 'reference_supporting_text',
       'reference_uri', 'subject_id'],
      dtype='object')
* This is the first record:
    object_id property_description property_id  property_label  \
0  HGNC:28817                   NA  RO:0002434  interacts with   

                                property_uri reference_date  \
0  http://purl.obolibrary.org/obo/RO_0002434             NA   

                           reference_supporting_text reference_uri subject_id  
0 

In [13]:
monarch_network = monarch.read_connections("monarch_connections_v2020-11-15.csv")
monarch_edges = monarch.build_edges(monarch_network)
monarch_nodes = monarch.build_nodes(monarch_network)


* This is the size of the data structure: (3095, 7)
* These are the attributes: Index(['object_id', 'object_label', 'reference_id_list', 'relation_id',
       'relation_label', 'subject_id', 'subject_label'],
      dtype='object')
* This is the first record:
    object_id object_label  reference_id_list relation_id  relation_label  \
0  HGNC:28817       LRRC59                NaN  RO:0002434  interacts with   

  subject_id subject_label  
0   HGNC:633          AQP1  


In [14]:
monarch_edges = monarch.build_edges(monarch_network)



The function "build_edges()" is running...
df (3095, 9)

* This is the size of the edges file data structure: (3095, 9)
* These are the edges attributes: Index(['object_id', 'property_description', 'property_id', 'property_label',
       'property_uri', 'reference_date', 'reference_supporting_text',
       'reference_uri', 'subject_id'],
      dtype='object')
* This is the first record:
    object_id property_description property_id  property_label  \
0  HGNC:28817                   NA  RO:0002434  interacts with   

                                property_uri reference_date  \
0  http://purl.obolibrary.org/obo/RO_0002434             NA   

                           reference_supporting_text reference_uri subject_id  
0  This edge comes from the Monarch Knowledge Gra...            NA   HGNC:633  

The Monarch network edges are built and saved at: /home/karolis/Structured review/bioknowledge-reviewer/bioknowledge_reviewer/monarch/monarch_edges_v2020-11-16.csv


Finished build_edges()

In [15]:
monarch_nodes = monarch.build_nodes(monarch_network)


The function "build_nodes()" is running...
Number of concepts: 650
Number of nodes CURIEs: 21
List of nodes CURIEs: dict_keys(['HGNC', 'Coriell', 'MONDO', 'FlyBase', 'ENSEMBL', 'GO', 'HP', 'UBERON', 'RGD', 'ClinVarVariant', 'CL', 'MGI', 'REACT', 'dictyBase', 'dbSNPIndividual', 'ZFIN', 'WormBase', 'dbSNP', 'EFO', 'OMIM', 'NCBIGene'])

Adding BioThings annotation: gene name, synonyms, description...
symbols: 269
querying 1-269...done.
Finished.
2 input query terms found dup hits:
	[('AQP1', 2), ('Aqp1', 2)]
104 input query terms found no hit:
	['FlyBase:CG17664', 'Htt<tm2Detl>', 'ENSEMBL:ENSCAFG00000003102', 'ENSEMBL:ENSGALG00000005209', 'AVP
Pass "returnall=True" to return complete lists of duplicate or missing query terms.

* This is the size of the nodes file data structure: (650, 6)
* These are the nodes attributes: Index(['description', 'id', 'name', 'preflabel', 'semantic_groups',
       'synonyms'],
      dtype='object')
* This is the first record:
                               

In [18]:
# print type of objects
print('type edges:', type(monarch_edges))
print('type nodes:', type(monarch_nodes))
print()

# print objects sizes
print('len edges:', len(monarch_edges))
print('len nodes:', len(monarch_nodes))
print()

# print object attribute
print('attribute edges:', monarch_edges[0].keys())
print('attribute nodes:', monarch_nodes[0].keys())

type edges: <class 'list'>
type nodes: <class 'list'>

len edges: 3095
len nodes: 650

attribute edges: dict_keys(['subject_id', 'object_id', 'property_id', 'property_label', 'property_description', 'property_uri', 'reference_uri', 'reference_supporting_text', 'reference_date'])
attribute nodes: dict_keys(['id', 'semantic_groups', 'preflabel', 'name', 'synonyms', 'description'])


- Monarch network is returned as both digital object (`monarch_edges`, `monarch_nodes`) and CSV files at _**monarch/**_ (`monarch_edges_version.csv`, `monarch_nodes_version.csv`)

## Graph library
### Create the review knowledge graph
#### import graph

Tasks:

* Load Networks and calculate graph nodes
* Retrieve extra connectivity for the graph from Monarch
* Build the review graph

In [6]:
%%time
# load networks and calculate graph nodes
graph_nodes_list, reg_graph_edges = graph.graph_nodes(
    curation=curation_edges,
    monarch=monarch_edges,
    transcriptomics=rna_edges,
    regulation=reg_edges
)

# Monarch graph connectivity
## get Monarch edges
monarch_network_graph = monarch.extract_edges(graph_nodes_list)
print('network: ',len(monarch_network_graph))

## save Monarch network
monarch.print_network(monarch_network_graph, 'monarch_connections_graph')

## build Monarch network with graph schema
monarch_graph_edges = monarch.build_edges(monarch_network_graph)
monarch_graph_nodes = monarch.build_nodes(monarch_network_graph)

# build review graph
edges = graph.build_edges(
    curation=curation_edges,
    monarch=monarch_graph_edges,
    transcriptomics=rna_edges,
    regulation=reg_graph_edges
)
nodes = graph.build_nodes(
    statements=edges,
    curation=curation_nodes,
    monarch=monarch_graph_nodes,
    transcriptomics=rna_nodes,
    regulation=reg_nodes
)


The function "graph_nodes()" is running...

Preparing networks...
Curated:
(2, 9)
Index(['subject_id', 'property_id', 'object_id', 'reference_uri',
       'reference_supporting_text', 'reference_date', 'property_label',
       'property_description', 'property_uri'],
      dtype='object')
Monarch:
(12221, 9)
Index(['object_id', 'property_description', 'property_id', 'property_label',
       'property_uri', 'reference_date', 'reference_supporting_text',
       'reference_uri', 'subject_id'],
      dtype='object')
Transcriptomics:
(3209, 9)
Index(['object_id', 'property_description', 'property_id', 'property_label',
       'property_uri', 'reference_date', 'reference_supporting_text',
       'reference_uri', 'subject_id'],
      dtype='object')
Regulatory:
(197267, 9)
Index(['object_id', 'property_description', 'property_id', 'property_label',
       'property_uri', 'reference_date', 'reference_supporting_text',
       'reference_uri', 'subject_id'],
      dtype='object')

Concatenating

100%|██████████| 9540/9540 [10:10:18<00:00,  5.19s/it]  



Finished extract_edges(). To save the retrieved Monarch edges use the function "print_network()".

network:  347873

Saving Monarch edges at: '/home/karolis/Structured review/bioknowledge-reviewer/bioknowledge_reviewer/monarch/monarch_connections_graph_v2020-11-20.csv'...


The function "build_edges()" is running...
df (347873, 9)

* This is the size of the edges file data structure: (347873, 9)
* These are the edges attributes: Index(['object_id', 'property_description', 'property_id', 'property_label',
       'property_uri', 'reference_date', 'reference_supporting_text',
       'reference_uri', 'subject_id'],
      dtype='object')
* This is the first record:
   object_id property_description property_id  property_label  \
0  HGNC:4544                   NA  RO:0002434  interacts with   

                                property_uri reference_date  \
0  http://purl.obolibrary.org/obo/RO_0002434             NA   

                           reference_supporting_text reference_uri subje

ValueError: You are trying to merge on float64 and object columns. If you wish to proceed you should use pd.concat

In [5]:
# obtain variables:
curation_edges = pd.read_csv("curation/data/HD/Empty_edges.csv")
curation_nodes = pd.read_csv("curation/data/HD/Empty_nodes.csv")

#monarch_network = monarch.read_connections("monarch_connections_v2020-11-15.csv")
#monarch_edges = monarch.build_edges(monarch_network)
#monarch_nodes = monarch.build_nodes(monarch_network)

#monarch_network_graph = monarch.read_connections("monarch_connections_graph_v2020-11-16.csv")
#monarch_graph_edges = monarch.build_edges(monarch_network_graph)
#monarch_graph_nodes = monarch.build_nodes(monarch_network_graph)

csv_path = './transcriptomics/GSE64810_mlhd_DESeq2_diffexp_DESeq2_outlier_trimmed_adjust.txt'
data = transcriptomics.read_data(csv_path)
clean_data = transcriptomics.clean_data(data)
data_edges = transcriptomics.prepare_data_edges(clean_data)
rna_network = transcriptomics.prepare_rna_edges(data_edges)

# build network with graph schema
rna_edges = transcriptomics.build_edges(rna_network)
rna_nodes = transcriptomics.build_nodes(rna_network)

gmt_path = './regulation/msigdb/data/c3.tft.v6.1.entrez.gmt'
regulation.prepare_msigdb_data(gmt_path)

# prepare individual networks
data = regulation.load_tf_gene_edges()
dicts = regulation.get_gene_id_normalization_dictionaries(data)
data_edges = regulation.prepare_data_edges(data, dicts)

# prepare regulation network
reg_network = regulation.prepare_regulation_edges(data_edges)

# build network with graph schema
reg_edges = regulation.build_edges(reg_network)
reg_nodes = regulation.build_nodes(reg_network)

#graph_nodes_list, reg_graph_edges = graph.graph_nodes(
#    curation=curation_edges,
#    monarch=monarch_edges,
#    transcriptomics=rna_edges,
#    regulation=reg_edges
#)



The function "read_data()" is running...

* This is the size of the raw expression data structure: (28087, 10)
* These are the expression attributes: Index(['Unnamed: 0', 'symbol', 'baseMean', 'HD.mean', 'Control.mean',
       'log2FoldChange', 'lfcSE', 'stat', 'pvalue', 'padj'],
      dtype='object')
* This is the first record:
           Unnamed: 0 symbol  baseMean    HD.mean  Control.mean  \
0  ENSG00000069011.10  PITX1  5.645675  18.684286      0.323793   

   log2FoldChange     lfcSE      stat        pvalue          padj  
0        4.769658  0.366367  13.01879  9.567529e-39  2.687232e-34  

The raw data is saved at: /home/karolis/Structured review/bioknowledge-reviewer/bioknowledge_reviewer/transcriptomics/HD/data/GSE64810_mlhd_DESeq2_diffexp_DESeq2_outlier_trimmed_adjust.csv


Finished read_data().


The function "clean_data()" is running. Keeping only data with FC > 1.5 and FDR < 5% ...

* This is the size of the clean expression data structure: (3209, 6)
* These are the clean 



The function "build_edges()" is running...

* This is the size of the edges file data structure: (197267, 9)
* These are the edges attributes: Index(['object_id', 'property_description', 'property_id', 'property_label',
       'property_uri', 'reference_date', 'reference_supporting_text',
       'reference_uri', 'subject_id'],
      dtype='object')
* This is the first record:
   object_id property_description property_id  property_label  \
0  HGNC:8803                   NA  RO:0002434  interacts with   

                                property_uri reference_date  \
0  http://purl.obolibrary.org/obo/RO_0002434     2007-01-01   

                           reference_supporting_text  \
0  This edge comes from the TRED dataset in "tfta...   

                                  reference_uri subject_id  
0  https://www.ncbi.nlm.nih.gov/pubmed/17202159  HGNC:8615  

The regulation network edges are built and saved at: /home/karolis/Structured review/bioknowledge-reviewer/bioknowledge_revie

In [7]:
# fix curation data:
curation_nodes = curation_nodes.astype('object')
#curation_nodes.loc[0]

In [8]:
curation_nodes['name'] = ["NaN", "NaN"]

In [9]:
curation_nodes.loc[0]

id                 NaN
semantic_groups    NaN
preflabel          NaN
synonyms           NaN
description        NaN
name               NaN
Name: 0, dtype: object

In [10]:
# construct df's
edges = graph.build_edges(
    curation=curation_edges,
    monarch=monarch_graph_edges,
    transcriptomics=rna_edges,
    regulation=reg_graph_edges
)
nodes = graph.build_nodes(
    statements=edges,
    curation=curation_nodes,
    monarch=monarch_graph_nodes,
    transcriptomics=rna_nodes,
    regulation=reg_nodes
)


The function "build_edges()" is running...

Preparing networks...
Curated:
(2, 9)
Index(['subject_id', 'property_id', 'object_id', 'reference_uri',
       'reference_supporting_text', 'reference_date', 'property_label',
       'property_description', 'property_uri'],
      dtype='object')
Monarch:
(347873, 9)
Index(['object_id', 'property_description', 'property_id', 'property_label',
       'property_uri', 'reference_date', 'reference_supporting_text',
       'reference_uri', 'subject_id'],
      dtype='object')
Transcriptomics:
(3209, 9)
Index(['object_id', 'property_description', 'property_id', 'property_label',
       'property_uri', 'reference_date', 'reference_supporting_text',
       'reference_uri', 'subject_id'],
      dtype='object')
Regulatory:
(13412, 9)
Index(['subject_id', 'property_id', 'object_id', 'reference_uri',
       'reference_supporting_text', 'reference_date', 'property_label',
       'property_description', 'property_uri'],
      dtype='object')

Concatenating

In [2]:
nodes = pd.read_csv("/home/karolis/Structured review/bioknowledge-reviewer/bioknowledge_reviewer/graph/graph_nodes_v2020-11-20.csv")
edges = pd.read_csv("/home/karolis/Structured review/bioknowledge-reviewer/bioknowledge_reviewer/graph/graph_edges_v2020-11-20.csv")

/home/karolis/anaconda3/envs/sr/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (3,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


- Regulation edges _merged_ with the graph is returned as both digital object (`reg_graph_edges`) and CSV file at _**graph/**_ (`regulation_graph_edges_version.csv`)
- Monarch network is returned as both digital object (`monarch_graph_edges`, `monarch_graph_nodes`) and CSV files at _**monarch/**_ (`monarch_edges_version.csv`, `monarch_nodes_version.csv`) overwritten the previous one.
- Review knowledge graph is returned as both digital object (`edges`, `nodes`) and CSV files at _**graph/**_ (`graph_edges_version.csv`, `graph_nodes_version.csv`)

In [3]:
# print type of objects
print('type edges:', type(edges))
print('type nodes:', type(nodes))
print()

# print objects sizes
print('len edges:', len(edges))
print('len nodes:', len(nodes))
print()

# print object attribute
print('attribute edges:', edges.columns)
print('attribute nodes:', nodes.columns)

type edges: <class 'pandas.core.frame.DataFrame'>
type nodes: <class 'pandas.core.frame.DataFrame'>

len edges: 364495
len nodes: 9540

attribute edges: Index(['subject_id', 'property_id', 'object_id', 'reference_uri',
       'reference_supporting_text', 'reference_date', 'property_label',
       'property_description', 'property_uri'],
      dtype='object')
attribute nodes: Index(['id', 'semantic_groups', 'preflabel', 'synonyms', 'name',
       'description'],
      dtype='object')


## Neo4jlib library
### Import the graph into Neo4j graph database
#### import neo4jlib

Tasks:

- Create Neo4j server instance
- Import review graph into the Neo4j graph database

In [56]:
%%time
# create a Neo4j server instance
neo4j_dir = neo4jlib.create_neo4j_instance('4.2.0')
print('The name of the neo4j directory is {}'.format(neo4j_dir))

# import to graph database
## prepare the graph to neo4j format
edges_df = utils.get_dataframe(edges)
nodes_df = utils.get_dataframe(nodes)
statements = neo4jlib.get_statements(edges_df)
concepts = neo4jlib.get_concepts(nodes_df)
print('statements: ', len(statements))
print('concepts: ',len(concepts))

## save files into neo4j import dir
neo4j_path = './{}'.format(neo4j_dir)
neo4jlib.save_neo4j_files(statements, neo4j_path, file_type = 'statements')
neo4jlib.save_neo4j_files(concepts, neo4j_path, file_type = 'concepts')

## import graph into neo4j database
neo4jlib.do_import(neo4j_path)

Creating a Neo4j community v4.2.0 server instance...
Preparing the server...
Configuration adjusted!
Starting the server...
Neo4j v4.2.0 is NOT running. Some problem occurred and should be checked. Bye!
The name of the neo4j directory is neo4j-community-4.2.0
statements:  364495
concepts:  9540

File './neo4j-community-4.2.0/import/ngly1/ngly1_statements.csv' saved.

File './neo4j-community-4.2.0/import/ngly1/ngly1_concepts.csv' saved.

The function "do_import()" is running...

The graph is imported into the server. Neo4j is NOT running. Some problem occurred and should be checked.You can start exploring and querying for hypothesis. If you change ports or authentication in the Neo4j configuration file, the hypothesis-generation modules performance, hypothesis.py and summary.py, will be affected.

CPU times: user 5.1 s, sys: 342 ms, total: 5.45 s
Wall time: 34.4 s


In [57]:
# print type of objects
print('type edges:', type(statements))
print('type nodes:', type(concepts))
print()

# print objects sizes
print('len edges:', len(statements))
print('len nodes:', len(concepts))
print()

# print object attribute
print('attribute edges:', statements.columns)
print('attribute nodes:', concepts.columns)

type edges: <class 'pandas.core.frame.DataFrame'>
type nodes: <class 'pandas.core.frame.DataFrame'>

len edges: 364495
len nodes: 9540

attribute edges: Index([':START_ID', ':TYPE', ':END_ID', 'reference_uri',
       'reference_supporting_text', 'reference_date', 'property_label',
       'property_description:IGNORE', 'property_uri'],
      dtype='object')
attribute nodes: Index(['id:ID', ':LABEL', 'preflabel', 'synonyms:IGNORE', 'name',
       'description'],
      dtype='object')


## hypothesis-generation library
### Query the graph for mechanistic explanation, then summarize the extracted paths
#### import hypothesis, summary


Tasks:

* Retrieve orthopheno paths with the `query` method.
* Retrieve orthopheno paths using relaxing node degree parameters with the `query` method.
* Retrieve orthopheno paths from a more open query topology with the `open_query` method.
* Get hypothesis summaries

### Ortopheno query with general nodes/relations removed

In [58]:
from neo4j import GraphDatabase
import sys,os
import json
import yaml
import datetime
import neo4j.exceptions

In [59]:
def parse_path( path ):
    """
    This function parses neo4j results.
    :param path: neo4j path object
    :return: parsed path dictionary
    """

    out = {}
    out['Nodes'] = []
    for node in path['path'].nodes:
        n = {}
        n['idx'] = node.id
        n['label'] = list(node.labels)[0]
        n['id'] = node.get('id')
        n['preflabel'] = node.get('preflabel')
        n['name'] = node.get('name')
        n['description'] = node.get('description')
        out['Nodes'].append(n)
    out['Edges'] = []
    for edge in path['path'].relationships:
        e = {}
        e['idx'] = edge.id
        e['start_node'] = edge.start_node.id
        e['end_node'] = edge.end_node.id
        e['type'] = edge.type
        e['preflabel'] = edge.get('property_label')
        e['references'] = edge.get('reference_uri')
        out['Edges'].append(e)
    return out

In [63]:
def get_node(source, target, port='7687'):
    """
    This function checks if a node exists within the graph
    """
    try:
        driver = GraphDatabase.driver("bolt://localhost:{}".format(port), auth=("neo4j", "ngly1"))
    except neo4j.exceptions.ServiceUnavailable:
        raise
    outputAll = list()
    with driver.session() as session:
        query = """MATCH (source { id: '""" + source + """' }), (target { id: '""" + target +  """' }), p = allShortestPaths((source)-[*..15]-(target)) RETURN p"""
        result = session.run(query)
        print(result)
        x = []
        for record in result:
            #path_dct = parse_path(record)
            x.append(record)
        print(x)
        return x, result

In [64]:
x, result = get_node('HGNC:4851', 'HGNC:18229')

[<Record p=<Path start=<Node id=477 labels=frozenset({'GENE'}) properties={'preflabel': 'HTT', 'name': 'huntingtin', 'description': "Huntingtin is a disease gene linked to Huntington's disease, a neurodegenerative disorder characterized by loss of striatal neurons. This is thought to be caused by an expanded, unstable trinucleotide repeat in the huntingtin gene, which translates as a polyglutamine repeat in the protein product. A fairly broad range of trinucleotide repeats (9-35) has been identified in normal controls, and repeat numbers in excess of 40 have been described as pathological. The huntingtin locus is large, spanning 180 kb and consisting of 67 exons. The huntingtin gene is widely expressed and is required for normal development. It is expressed as 2 alternatively polyadenylated forms displaying different relative abundance in various fetal and adult tissues. The larger transcript is approximately 13.7 kb and is expressed predominantly in adult and fetal brain whereas the s

In [65]:
x[1]

<Record p=<Path start=<Node id=477 labels=frozenset({'GENE'}) properties={'preflabel': 'HTT', 'name': 'huntingtin', 'description': "Huntingtin is a disease gene linked to Huntington's disease, a neurodegenerative disorder characterized by loss of striatal neurons. This is thought to be caused by an expanded, unstable trinucleotide repeat in the huntingtin gene, which translates as a polyglutamine repeat in the protein product. A fairly broad range of trinucleotide repeats (9-35) has been identified in normal controls, and repeat numbers in excess of 40 have been described as pathological. The huntingtin locus is large, spanning 180 kb and consisting of 67 exons. The huntingtin gene is widely expressed and is required for normal development. It is expressed as 2 alternatively polyadenylated forms displaying different relative abundance in various fetal and adult tissues. The larger transcript is approximately 13.7 kb and is expressed predominantly in adult and fetal brain whereas the sm

In [67]:
result.consume()

In [63]:
def query_shortest_paths(source, target, max_length=4, port='7687'):
    """
    This function gets the shortest paths between source and
    target. max_length determines the maximum path size allowed.
    """
    try:
        driver = GraphDatabase.driver("bolt://localhost:{}".format(port), auth=("neo4j", "ngly1"))
    except neo4j.exceptions.ServiceUnavailable:
        raise
    outputAll = list()
    with driver.session() as session:
        print("kaas")
        query = """MATCH (source { id: '""" + source + """' }), (target { id: '""" + target +  """' }), p = shortestPath((source)-[*..15]-(target)) RETURN p"""
        result = session.run(query)
        print(result)
        return result

In [64]:
seed = list([
        'HGNC:4851',  # Htt human gene
        'HGNC:18229'  # Rhes human gene
])
result = query_shortest_paths(seed[0], seed[1], max_length=999)

kaas


In [65]:
summary = result.consume()

In [66]:
summary.query

"MATCH (source { id: 'HGNC:4851' }), (target { id: 'HGNC:18229' }), p = shortestPath((source)-[*..15]-(target)) RETURN p"

In [68]:
result.data()

[]

In [14]:
%%time
# get orthopheno paths
seed = list([
        'HGNC:4851',  # Htt human gene
        'HGNC:18229'  # Rhes human gene
])
hypothesis.query(seed,queryname='Htt_Rhes',port='7687') 


The function "query()" is running...

The query results are saved at: /home/karolis/Structured review/bioknowledge-reviewer/bioknowledge_reviewer/hypothesis/query_Htt_Rhes_pwdl50_phdl20_paths_v2020-11-23.json


The query results are saved at: /home/karolis/Structured review/bioknowledge-reviewer/bioknowledge_reviewer/hypothesis/query_Htt_Rhes_pwdl50_phdl20_paths_v2020-11-23.json




Hypothesis generator has finished. 2 QUERIES completed.


CPU times: user 19.3 ms, sys: 4.5 ms, total: 23.8 ms
Wall time: 263 ms


In [38]:
%%time
# get orthopheno paths relaxing pathway and phenotype node degrees
seed = list([
        'HGNC:4851',  # NGLY1 human gene
        'HGNC:18229'  # AQP1 human gene
])
hypothesis.query(seed, queryname='Htt_Rhes', pwdegree='1000', phdegree='1000', port='7687')


The function "query()" is running...

The query results are saved at: /home/karolis/Structured review/bioknowledge-reviewer/bioknowledge_reviewer/hypothesis/query_Htt_Rhes_pwdl1000_phdl1000_paths_v2020-11-17.json


The query results are saved at: /home/karolis/Structured review/bioknowledge-reviewer/bioknowledge_reviewer/hypothesis/query_Htt_Rhes_pwdl1000_phdl1000_paths_v2020-11-17.json




Hypothesis generator has finished. 2 QUERIES completed.


CPU times: user 28.4 ms, sys: 0 ns, total: 28.4 ms
Wall time: 379 ms


In [39]:
%%time
# get orthopheno paths from a more open query topogology
seed = list([
        'HGNC:4851',  # NGLY1 human gene
        'HGNC:18229'  # AQP1 human gene
])
hypothesis.open_query(seed,queryname='Htt_Rhes',port='7687')


The function "open_query()" is running...

The query results are saved at: /home/karolis/Structured review/bioknowledge-reviewer/bioknowledge_reviewer/hypothesis/query_Htt_Rhes_paths_v2020-11-17.json


The query results are saved at: /home/karolis/Structured review/bioknowledge-reviewer/bioknowledge_reviewer/hypothesis/query_Htt_Rhes_paths_v2020-11-17.json




Hypothesis generator has finished. 2 QUERIES completed.


CPU times: user 18.8 ms, sys: 7.78 ms, total: 26.6 ms
Wall time: 850 ms


In [32]:
import hypothesis

In [33]:
%%time
seed = list([
        'HGNC:4851',  # NGLY1 human gene
        'HGNC:18229'  # AQP1 human gene
])
r = hypothesis.shortest_paths(seed[0], seed[1], max_length=50, port='7687')

CPU times: user 4.27 ms, sys: 2 µs, total: 4.27 ms
Wall time: 23.3 ms


In [34]:
print(r)

[{'source': 'HGNC:4851', 'target': 'HGNC:18229', 'paths': []}]


In [4]:
%%time
# get summary
data = summary.path_load('./hypothesis/query_ngly1_aqp1_pwdl1000_phdl1000_paths_v2020-09-14.json')

# parse data for summarization
data_parsed = list()
for query in data:
    query_parsed = summary.query_parser(query)
    data_parsed.append(query_parsed)
summary.metapaths(data_parsed)
summary.nodes(data_parsed)
summary.node_types(data_parsed)
summary.edges(data_parsed)
summary.edge_types(data_parsed)


The function "query_parser()" is running...

Finished query_parser().


The function "query_parser()" is running...

Finished query_parser().


The function "metapaths()" is running...

Printing summaries...


File '/home/karolis/Structured review/bioknowledge-reviewer/bioknowledge_reviewer/summaries/query_source:HGNC:17646_target:HGNC:633_summary_metapaths_v2020-09-21.csv' saved.

File '/home/karolis/Structured review/bioknowledge-reviewer/bioknowledge_reviewer/summaries/query_source:HGNC:17646_target:HGNC:633_summary_entities_in_metapaths_v2020-09-21.csv' saved.

Printing summaries...


File '/home/karolis/Structured review/bioknowledge-reviewer/bioknowledge_reviewer/summaries/query_source:HGNC:633_target:HGNC:17646_summary_metapaths_v2020-09-21.csv' saved.

File '/home/karolis/Structured review/bioknowledge-reviewer/bioknowledge_reviewer/summaries/query_source:HGNC:633_target:HGNC:17646_summary_entities_in_metapaths_v2020-09-21.csv' saved.

Finished metapaths().


The function "nod